In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from ISLP.models import (ModelSpec as MS,
                         summarize)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [35]:
Spotify = pd.read_csv('/Users/Camer/Downloads/Spring 2025 Classes/Stat 411/Projects/Project 2/Datasets/data.csv')
Spotify.head(5)

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [36]:
Spotify.describe()

,valence,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo
count,170653.000000,170653.000000,170653.000000,170653.000000,1.706530e+05,170653.000000,170653.000000,170653.000000,170653.000000,170653.000000,170653.000000,170653.000000,170653.000000,170653.000000,170653.000000
mean,0.528587,1976.787241,0.502115,0.537396,2.309483e+05,0.482389,0.084575,0.167010,5.199844,0.205839,-11.467990,0.706902,31.431794,0.098393,116.861590
std,0.263171,25.917853,0.376032,0.176138,1.261184e+05,0.267646,0.278249,0.313475,3.515094,0.174805,5.697943,0.455184,21.826615,0.162740,30.708533
min,0.000000,1921.000000,0.000000,0.000000,5.108000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000
25%,0.317000,1956.000000,0.102000,0.415000,1.698270e+05,0.255000,0.000000,0.000000,2.000000,0.098800,-14.615000,0.000000,11.000000,0.034900,93.421000
50%,0.540000,1977.000000,0.516000,0.548000,2.074670e+05,0.471000,0.000000,0.000216,5.000000,0.136000,-10.580000,1.000000,33.000000,0.045000,114.729000
75%,0.747000,1999.000000,0.893000,0.668000,2.624000e+05,0.703000,0.000000,0.102000,8.000000,0.261000,-7.183000,1.000000,48.000000,0.075600,135.537000
max,1.000000,2020.000000,0.996000,0.988000,5.403500e+06,1.000000,1.000000,1.000000,11.000000,1.000000,3.855000,1.000000,100.000000,0.970000,243.507000


In [39]:
Spotify.columns

Index(['valence', 'year', 'acousticness', 'artists', 'danceability',
       'duration_ms', 'energy', 'explicit', 'id', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'name', 'popularity', 'release_date',
       'speechiness', 'tempo', 'funness', 'sentiment'],
      dtype='object')

In [46]:
# Define the response and predictor variables
response = 'popularity'
predictors = ['valence', 'acousticness', 'danceability', 'energy', 'instrumentalness',
              'liveness', 'loudness', 'speechiness', 'explicit', 'tempo']

# Create a dataframe of the predictor variables
X = Spotify[predictors]

# Create a dataframe for the response variable
y = Spotify[response]

# Adding intercept to predictor dataframe
X = sm.add_constant(X)

In [47]:
# Create our training dataset and our test dataset splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 7)

# Creates our model based on the data in the training dataset
model = sm.OLS(y_train, X_train).fit()

# Runs the predictions from our model created on the training set on the test set
predictions = model.predict(X_test)

# Computes the MSE for our predictions
mse = mean_squared_error(y_test, predictions)

# Computes the RSE for our predictions
n = len(y_test)
p = X_train.shape[1] - 1
rse = np.sqrt(mse * n / (n - p - 1))

# Computes the R-squared for our predictions
r_squared = r2_score(y_test, predictions)

print(f'Test MSE: {mse:.3f}')
print(f'RSE: {rse:.3f}')
print(f'R-Squared: {r_squared:.3f}')

Test MSE: 261.683
RSE: 16.179
R-Squared: 0.454


My first MLR model with no additional interaction terms added or removing any unnecessary terms didn't perform to well, I will now try adding interaction terms between predictors like energy, loudness, and danceability that should be correlated and remove variables that are not needed potentially.

In [56]:
# Creating first submodel of MLR
X1 = Spotify[['energy', 'danceability', 'loudness', 'liveness', 'valence', 'explicit',
              'tempo']]
cv1 = -cross_val_score(LinearRegression(), X1, y, scoring = 'neg_mean_squared_error', cv = 3).mean()

# Creating second submodel of MLR
X2 = Spotify[['instrumentalness', 'speechiness', 'acousticness']]
cv2 = -cross_val_score(LinearRegression(), X2, y, scoring = 'neg_mean_squared_error', cv = 3).mean()

# Creating third submodel of MLR
X3 = Spotify[['valence', 'explicit', 'tempo']]
cv3 = -cross_val_score(LinearRegression(), X3, y, scoring = 'neg_mean_squared_error', cv = 3).mean()

if cv1 > cv2 and cv1 > cv3:
    bestX = X1
    print('Best model: X1')
elif cv2 > cv1 and cv2 > cv3:
    bestX = X2
    print('Best model: X2')
elif cv3 > cv1 and cv3 > cv2:
    bestX = X3
    print('Best model: X3')

Best model: X3


In [55]:
model2 = sm.OLS(y, sm.add_constant(bestX)).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:             popularity   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.054
Method:                 Least Squares   F-statistic:                     3239.
Date:                Thu, 10 Apr 2025   Prob (F-statistic):               0.00
Time:                        00:24:35   Log-Likelihood:            -7.6357e+05
No. Observations:              170653   AIC:                         1.527e+06
Df Residuals:                  170649   BIC:                         1.527e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         19.4331      0.215     90.501      0.0